> Задача 3. Доработка метода для оценки эксперимента с применением постстратификации
  

Реализуйте функцию для оценки эксперимента с применением постстратификации.

Доработайте метод _ttest_strat класса ExperimentsService.

In [ ]:
import numpy as np
import pandas as pd
from pydantic import BaseModel
from scipy import stats


class Design(BaseModel):
    """Дата-класс с описание параметров эксперимента.

    statistical_test - тип статтеста. ['ttest', 'bootstrap']
    effect - размер эффекта в процентах
    alpha - уровень значимости
    beta - допустимая вероятность ошибки II рода
    stratification - постстратификация. 'on' - использовать постстратификация, 'off - не использовать.
    """
    statistical_test: str = 'ttest'
    effect: float = 3.
    alpha: float = 0.05
    beta: float = 0.1
    stratification: str = 'off'


class ExperimentsService:

    def _ttest_strat(self, metrics_strat_a_group, metrics_strat_b_group):
        """Применяет постстратификацию, возвращает pvalue.

        Веса страт считаем по данным обеих групп.
        Предполагаем, что эксперимент проводится на всей популяции.
        Веса страт нужно считать по данным всей популяции.

        :param metrics_strat_a_group (np.ndarray): значения метрик и страт группы A.
            shape = (n, 2), первый столбец - метрики, второй столбец - страты.
        :param metrics_strat_b_group (np.ndarray): значения метрик и страт группы B.
            shape = (n, 2), первый столбец - метрики, второй столбец - страты.
        :param design (Design): объект с данными, описывающий параметры эксперимента
        :return (float): значение p-value
        """
        # YOUR_CODE_HERE
        df_a = pd.DataFrame(metrics_strat_a_group, columns=['metric', 'strat'])
        df_b = pd.DataFrame(metrics_strat_b_group, columns=['metric', 'strat'])
        df = pd.concat([df_a, df_b])
        weights = df['strat'].value_counts(normalize=True).to_dict()
        strat_mean_a = df_a.groupby('strat')['metric'].mean()
        strat_mean_b = df_b.groupby('strat')['metric'].mean()

        df_means_weights_a = pd.merge(
            strat_mean_a,
            pd.Series(weights, name='weight'),
            how='inner',
            left_index=True,
            right_index=True
        )

        df_means_weights_b = pd.merge(
            strat_mean_b,
            pd.Series(weights, name='weight'),
            how='inner',
            left_index=True,
            right_index=True
        )

        df_means_weights_a['weight'] = df_means_weights_a['weight'] / df_means_weights_a['weight'].sum()
        mean_strat_a = (df_means_weights_a['weight'] * df_means_weights_a['metric']).sum()

        df_means_weights_b['weight'] = df_means_weights_b['weight'] / df_means_weights_b['weight'].sum()
        mean_strat_b = (df_means_weights_b['weight'] * df_means_weights_b['metric']).sum()

        strat_vars_a = df_a.groupby('strat')['metric'].var()
        df_vars_weights_a = pd.merge(
            strat_vars_a,
            pd.Series(weights, name='weight'),
            how='inner',
            left_index=True,
            right_index=True
        )

        strat_vars_b = df_b.groupby('strat')['metric'].var()
        df_vars_weights_b = pd.merge(
            strat_vars_b,
            pd.Series(weights, name='weight'),
            how='inner',
            left_index=True,
            right_index=True
        )

        df_vars_weights_a['weight'] = df_vars_weights_a['weight'] / df_vars_weights_a['weight'].sum()
        var_strat_a = (df_vars_weights_a['weight'] * df_vars_weights_a['metric']).sum()

        df_vars_weights_b['weight'] = df_vars_weights_b['weight'] / df_vars_weights_b['weight'].sum()
        var_strat_b = (df_vars_weights_b['weight'] * df_vars_weights_b['metric']).sum()

        delta_mean_strat = mean_strat_b - mean_strat_a
        std_mean_strat = (var_strat_b / len(df_b) + var_strat_a / len(df_a)) ** 0.5
        t = delta_mean_strat / std_mean_strat
        pvalue = (1 - stats.norm.cdf(np.abs(t))) * 2

        return pvalue


    def get_pvalue(self, metrics_strat_a_group, metrics_strat_b_group, design):
        """Применяет статтест, возвращает pvalue.

        :param metrics_strat_a_group (np.ndarray): значения метрик и страт группы A.
            shape = (n, 2), первый столбец - метрики, второй столбец - страты.
        :param metrics_strat_b_group (np.ndarray): значения метрик и страт группы B.
            shape = (n, 2), первый столбец - метрики, второй столбец - страты.
        :param design (Design): объект с данными, описывающий параметры эксперимента
        :return (float): значение p-value
        """
        if design.statistical_test == 'ttest':
            if design.stratification == 'off':
                _, pvalue = stats.ttest_ind(metrics_strat_a_group[:, 0], metrics_strat_b_group[:, 0])
                return pvalue
            elif design.stratification == 'on':
                return self._ttest_strat(metrics_strat_a_group, metrics_strat_b_group)
            else:
                raise ValueError('Неверный design.stratification')
        else:
            raise ValueError('Неверный design.statistical_test')


if __name__ == '__main__':
    metrics_strat_a_group = np.zeros((10, 2,))
    metrics_strat_a_group[:, 0] = np.arange(10)
    metrics_strat_a_group[:, 1] = (np.arange(10) < 4).astype(float)
    metrics_strat_b_group = np.zeros((10, 2,))
    metrics_strat_b_group[:, 0] = np.arange(1, 11)
    metrics_strat_b_group[:, 1] = (np.arange(10) < 5).astype(float)
    design = Design(stratification='on')
    ideal_pvalue = 0.037056

    experiments_service = ExperimentsService()
    pvalue = experiments_service.get_pvalue(metrics_strat_a_group, metrics_strat_b_group, design)

    np.testing.assert_almost_equal(ideal_pvalue, pvalue, decimal=4, err_msg='Неверное значение pvalue')
    print('simple test passed')
